In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import json
import spotipy
import requests
from spotipy.oauth2 import SpotifyClientCredentials
from watson_developer_cloud import ToneAnalyzerV3

tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    iam_apikey='mZTAzzNSB4nU1VrkLwTflh-L_FAcSNOxM4IhB1t9BBZI',
    url='https://gateway-lon.watsonplatform.net/tone-analyzer/api')


client_credentials_manager = SpotifyClientCredentials(client_id='39ac75394bcd4eb0a2ad340bd41a0f39',
                                                      client_secret='7a131b3539a944c7bb4e8906aee877ed')

spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


list_of_url_by_years = [{"link":"0RQPCufJ8sFM0f6diGOqx2","year":"1989"},{"link":"2Ezq5tXBfcm08IgV6jVbD5","year":"1990"},
                        {"link":"4ASN0RlS651kVOnRsohaFL","year":"1991"},{"link":"3n98oIBshWorMOLJazIfJS","year":"1992"},
                        {"link":"45iNcb8ojtrQARtEpcpvvv","year":"1993"},{"link":"5NBp8N1KQBFipfxTJ9eYEe","year":"1994"},
                        {"link":"3KBYrJrYxfw9meQLwZcLh2","year":"1995"},{"link":"0e5LYEb4oqWeVCDhTKLCBQ","year":"1996"},
                        {"link":"2KizNGg9OiXSt2f8RyRGcr","year":"1997"},{"link":"62FCnGM83VWvtv6zAlMgTd","year":"1998"},
                        {"link":"7gTvhRcUZaXt8ydN3AAIqF","year":"1999"},{"link":"5FPS4YigpTkW5BReGWqQhf","year":"2000"},
                        {"link":"1js2rPNvOkRSFkyI5FnQ6e","year":"2001"},{"link":"4TG8HBGn9f7EbiV5cibtsj","year":"2002"},
                        {"link":"4HHSrrc1AChFvSdtN7SWKY","year":"2003"},{"link":"4INZgHh1xAfPjwRN7yaSvJ","year":"2004"},
                        {"link":"0jh6TBqviZXDFn1fZM8Wew","year":"2005"},{"link":"1wedGBMtddnGxpmBij5lUO","year":"2006"},
                        {"link":"22U6wJjyDlCYDzbJ6oIg3G","year":"2007"},{"link":"5risns7vjPjRJjwdRlqVQa","year":"2008"},
                        {"link":"78nme2PFmjzWHgUelXerSm","year":"2009"},{"link":"7fsFPe6ckDjWN673sk5Qzz","year":"2010"},
                        {"link":"42004WRNg430hY1cShpTsR","year":"2011"},{"link":"3umOgg13I8RjiT3QCB6ogo","year":"2012"},
                        {"link":"2kDPYiTtUtm5eZUkVYJ4f0","year":"2013"},{"link":"0KzjapF1zYpPYARZFeBnYm","year":"2014"},
                        {"link":"6MT7PxSJmrg8O31Z5vx1iJ","year":"2015"},{"link":"3HtBlISXM86gijSJ3zp1C6","year":"2016"},
                        {"link":"37i9dQZF1DX5nwnRMcdReF","year":"2017"},{"link":"4hOKQuZbraPDIfaGbM3lKI","year":"2018"}]

for index_elem,url_year in enumerate(list_of_url_by_years):
    
    pattern_url = re.compile(r'..external_urls..{.spotify.:.https:....open.spotify.com..track........................')
    pattern_externallUrl = re.compile(r'https:......open.spotify.com...track.........................')
    pattern3 = re.compile(r'popularity":...')

    url_byYear = urlopen("https://open.spotify.com/playlist/" + url_year["link"] )
    soup = BeautifulSoup(url_byYear.read())

    result = pattern_url.findall(str(soup))
    result = pattern_externallUrl.findall(str(result))
    result = re.sub('[\'\\\\\\\\]', '', str(result))
    result = re.sub('[,]', ' ', result)
    result = re.sub(r']', '\n', result)
    result = re.sub(r'\[', '\n', result)
    result = result.split()
    p  = pattern3.findall(str(soup))
    p2 = re.sub('[,]', '\n', str(p))

    pattern_indexUrl = re.compile(r'<meta property="twitter:audio:artist_name" content="..................................')
    api_key ='31a1860675cdd02bbef3b4032ee95a86'
    data = [{}]

    for index, elem in enumerate(result):
        url_byTrack = urlopen(elem)
        soup = BeautifulSoup(url_byTrack.read())
        track = soup.find_all('h1')
        track_item = str(track[0])    
        track_item = track_item[21:-12]
        artist_item = str(track[1])
        artist_item = artist_item[12:-5]       
        spotify_results = spotify.search(q='artist:' + artist_item, type='artist')
        items = spotify_results['artists']['items']
        url_lyrics = "https://api.musixmatch.com/ws/1.1/matcher.lyrics.get?q_track="+track_item+"&q_artist="+artist_item+"&apikey="+ api_key
        response = json.loads(requests.get(url_lyrics).text)
        if len(items) > 0 and response['message']['header']['status_code'] == 200:
            words = response['message']['body']['lyrics']["lyrics_body"]
            if len(words) > 0:      
                words = words[0:-70]
            else:
                words = 'none'
            tone_analysis = tone_analyzer.tone({'text': words},'application/json').get_result()
            if len(tone_analysis['document_tone']['tones']) > 0:  
                data.append({'index':index+1,'artist':artist_item,'genre': items[0]['genres'],'song':track_item,'year':url_year["year"],'song_lyrics':words,'lyrics_emotion':tone_analysis['document_tone']['tones'][0]['tone_name']})
            else:
                data.append({'index':index+1,'artist':artist_item,'genre': items[0]['genres'],'song':track_item,'year':url_year["year"],'song_lyrics':words,'lyrics_emotion':'none'})
        elif len(items) > 0:
            data.append({'index':index+1,'artist':artist_item,'genre': items[0]['genres'],'song':track_item,'year':url_year["year"],'song_lyrics':'none','lyrics_emotion':'none'})
        else:
            data.append({'index':index+1,'artist':artist_item,'genre': 'none','song':track_item,'year':url_year["year"],'song_lyrics':'none','lyrics_emotion':'none'})              
        

print(json.dumps(data))






0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
[{}, {"index": 1, "artist": "Mariah Carey", "genre": ["dance pop", "pop", "r&b", "urban contemporary"], "song": "We Belong Together", "year": "2005", "song_lyrics": "Sweet love yeah\nI didn't mean it when I said I didn't love you so\nI should have held on tight, I never should have let you go\nI didn't know nothing I was stupid\nI was foolish, I was lying to myself\nI couldn't have fathomed I would ever be without your love\nNever imagined I'd be sitting here beside myself\n'Cause I didn't know you, 'cause I didn't know me\nBut I thought I knew everyth

In [ ]:
import re
import json

data = {}
